In [39]:
from langchain_core.tools import tool

@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    return f"Weather in {location}: Sunny, 72°F"


@tool
def make_it_casual(text: str) -> str:
    """Change the given text to be more casual"""
    return f"Edited text: {text}"


In [40]:
from dotenv import load_dotenv
from langchain.agents import create_agent

load_dotenv()

tools = [get_weather, search, make_it_casual]

agent = create_agent(
    model="gpt-4o-mini",
    tools=tools
)

In [33]:

response = agent.invoke({"messages": [{'role': 'user', 'content': 'How is the weather in Seoul?'}]})


In [34]:
print(response)
print(response['messages'])
print(response['messages'][-1].content)

{'messages': [HumanMessage(content='How is the weather in Seoul?', additional_kwargs={}, response_metadata={}, id='8fe43690-892a-4b59-84d0-aad1900ae376'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 68, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-Cd4nHDBtvSOuDHiEHUWBU0kgtNnEC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2d3a4e40-e45d-4488-bb99-2c1df66e1901-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Seoul'}, 'id': 'call_5qAJaYeJHrRFArUnys6GtQHh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 68, 'output_tokens': 15, 'total_t

In [42]:
for chunk in agent.stream({
    "messages": [{"role": "user", "content": "서울 날씨 캐주얼한 톤으로 알려줘"}]
}, stream_mode="values"):
    # Each chunk contains the full state at that point
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

Agent: 서울 날씨 캐주얼한 톤으로 알려줘
Calling tools: ['get_weather']
Agent: Weather in 서울: Sunny, 72°F
Calling tools: ['make_it_casual']
Agent: Edited text: 서울 날씨는 맑고 기온은 22도예요.
Agent: 서울 날씨는 맑고, 기온은 22도야! 좋네~
